In [9]:
import pandas as pd

In [45]:
skincare=pd.read_csv('SkinCareDetail.csv')
makeup=pd.read_csv('MakeupDetail.csv')
skincare.shape,makeup.shape

((4397, 11), (4896, 11))

In [46]:
#Combine makeup and skincare data
skincare.insert(8,'Main_Category',["Skincare"]*4397,True)
makeup.insert(8,'Main_Category',["Makeup"]*4896,True)
ulta=pd.concat([skincare,makeup],axis=0)
ulta.index=range(0,len(ulta))
ulta['Price']=[x.split("Original Price")[0] for x in ulta['Price']]
ulta.sample(3)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Category,Details,Image
6495,Tweezerman,Eyelash Guard & Comb,3.9,$12.00,https://www.ulta.com/p/eyelash-guard-comb-pimp...,2591902,182.0,Ingredients are not available,Makeup,Makeup Brushes & Tools,Details\nFeatures\nLash guide perfectly lays a...,https://media.ulta.com/i/ulta/2098?w=582&h=582...
1339,BYOMA,Moisturizing Rich Cream,4.5,$15.99,https://www.ulta.com/p/moisturizing-rich-cream...,2601333,202.0,"Aqua, Glycerin, Caprylic/Capric Triglyceride, ...",Skincare,Moisturizers,Details\nBenefits\nRich daily moisturizing cre...,https://media.ulta.com/i/ulta/2601333?w=78&h=78
2764,Exuviance,Daily Resurfacing Leave On Face Peel,4.2,$75.50,https://www.ulta.com/p/daily-resurfacing-leave...,2554745,165.0,"Aqua/water/eau, Citric Acid, Arginine, Ammoniu...",Skincare,Treatment & Serums,Details\nBenefits\nNow you can safely and easi...,https://media.ulta.com/i/ulta/2554745?w=78&h=78


In [47]:
indexAge=ulta[ ulta['Category'].isin(['Makeup Brushes & Tools' ,'Makeup Bags & Organizers','Skincare Tools',
                                     'Bath & Body Accessories'
                                     ])].index
ulta.drop(indexAge,inplace=True)

In [53]:
#Drop lines with <3 ingredients provided
ulta['Ig_list']=ulta.apply(lambda x:x['Ingredient'].split(','),axis=1)
ulta['len']=ulta.apply(lambda x:len(x['Ig_list']),axis=1)
indexAge=ulta[ ulta['len']<3].index
ulta.drop(indexAge,inplace=True)
ulta = ulta.drop(columns=['Ig_list', 'len'])

In [54]:
#Drop duplicates
ulta.drop_duplicates(subset=["Name",'ID'],inplace=True)
ulta.to_csv('Clean_data',index=None)

In [6]:
from platform import python_version

print(python_version())

3.9.13
